In [1]:
#this is currency conversion project that convert with latest conversion rate

In [15]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated
from langchain_core.tools import tool
import requests
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
import json

In [2]:
api_key = os.getenv("EXCHANGE_RATE_API_KEY")

In [3]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/{api_key}/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [4]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [5]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1753056001,
 'time_last_update_utc': 'Mon, 21 Jul 2025 00:00:01 +0000',
 'time_next_update_unix': 1753142401,
 'time_next_update_utc': 'Tue, 22 Jul 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 86.2303}

In [6]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [8]:
# tool binding
llm = ChatOpenAI()

In [9]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [11]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [12]:
ai_message = llm_with_tools.invoke(messages)

In [13]:
messages.append(ai_message)

In [14]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'call_au3LX82fDkUrm6Crr1a76SJh',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'call_Td87VPT9XSCvE2VUgcW5M1D9',
  'type': 'tool_call'}]

In [16]:
for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)

In [17]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_au3LX82fDkUrm6Crr1a76SJh', 'function': {'arguments': '{"base_currency": "INR", "target_currency": "USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_Td87VPT9XSCvE2VUgcW5M1D9', 'function': {'arguments': '{"base_currency_value": 10}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 123, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BvnLq000c03zowT0YBcsqctyUUljQ', 'service_tier': 'defa

In [18]:
llm_with_tools.invoke(messages).content

'The conversion factor between INR and USD is 0.0116. \n\nTherefore, when you convert 10 INR to USD, the equivalent amount is approximately 0.116 USD.'